Answers
-------
1. Both are weight initialization techniques that attempt to normalize the initial weights of a NN. The purpose 
  of it is to speed training and prevent vanishing and exploding gradients. This problem is due to the large variance
  between inputs and outputs of layers which these weight initialization techniques attempt to fix.

2. No, this would defeat the purpose of the initialization. The network won't be able to learn since there will be 
  no variability between the weight values - no breaking symmetry. This means that all neurons will always output
  the same weights (symmetry) which will be equivalent to training a network with a single neuron per layer.

3. It's ok to initialize bias to 0. Does not make a difference.

4. Depending on the problem and the other techniques used for training:
  - Sigmoid and TanH
    * Since these are sensitive to vanishing gradients, it's important to use them with batch normalization.
    * Also, they are more expensive to compute, therefore, they could slow down larger networks
  - ReLU
    * Fast but has the risk of creating dead neurons (their input is less than zero and they output 0).
    * Leaky ReLU fixes this problem
    * Good for smaller networks
  - Swish
    * Good activation function for larger networks

5. If we set the momentum hyperparameter b closer to 1, the momentum of previous weights will get taken into account more.
  It will put less emphasis on the current weight updates. This can make the weight updates to accumulate speed to the point
  that it overshoots the minimum.

6. 3 ways to produce a sparse model:
  - l1 regularization
  - Dropout
  - ??

7. Dropout does slow down training (usually by a factor of 2). No effects on inference as long as we do the dropout
  scaling during training (we can also do it during inference). In MC, inference is affected as well.

In [79]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [52]:
# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the pixel values to a range between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

### Exercise 8.a

In [77]:
def build_dnn_model(input_shape, num_hidden_layers, num_neurons_hidden, 
                    output_num, optimizer, loss, use_batch_norm=False, use_dropout=False):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=input_shape))
  
  for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_neurons_hidden, kernel_initializer="he_normal"))
    if use_dropout and num_hidden_layers - i < 3:
      model.add(tf.keras.layers.Dropout(rate=0.25))    
    if use_batch_norm: 
      model.add(tf.keras.layers.BatchNormalization())    
    model.add(tf.keras.layers.Activation("swish"))

  if use_batch_norm: 
    model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Dense(output_num, activation="softmax"))
  model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

  return model

def fit_dnn_model(model, X_train, y_train, epochs=20, validation_split=0.2, callbacks=[]):
  return model.fit(X_train, y_train, 
            epochs=epochs, 
            validation_split=validation_split, 
            callbacks=callbacks)



### Exercise 8.b

In [69]:
input_shape = list(X_train.shape[1:])
optimizer = tf.keras.optimizers.legacy.Nadam()
model = build_dnn_model(input_shape, 20, 100, 10, optimizer, "sparse_categorical_crossentropy")

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model", save_best_only=True)

history = fit_dnn_model(model, X_train, y_train, epochs=20, validation_split=0.2, callbacks=[early_stopping_cb, model_checkpoint_cb])

Epoch 1/20
1249/1250 [============================>.] - ETA: 0s - loss: 2.0768 - accuracy: 0.2063INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 8s 5ms/step - loss: 2.0768 - accuracy: 0.2064 - val_loss: 2.0747 - val_accuracy: 0.2234
Epoch 2/20
1247/1250 [============================>.] - ETA: 0s - loss: 1.8807 - accuracy: 0.2945INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 6s 5ms/step - loss: 1.8810 - accuracy: 0.2944 - val_loss: 1.8324 - val_accuracy: 0.3160
Epoch 3/20
1242/1250 [============================>.] - ETA: 0s - loss: 1.8289 - accuracy: 0.3160INFO:tensorflow:Assets written to: my_cifar10_model/assets


...

1250/1250 [==============================] - 6s 5ms/step - loss: 1.7221 - accuracy: 0.3751 - val_loss: 1.7227 - val_accuracy: 0.3755
Epoch 10/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.7063 - accuracy: 0.3815 - val_loss: 1.7380 - val_accuracy: 0.3663
Epoch 11/20
1250/1250 [==============================] - 6s 5ms/step - loss: 1.6947 - accuracy: 0.3823 - val_loss: 1.7972 - val_accuracy: 0.3547
Epoch 12/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6843 - accuracy: 0.3947 - val_loss: 1.7237 - val_accuracy: 0.3863
Epoch 13/20
1246/1250 [============================>.] - ETA: 0s - loss: 1.6697 - accuracy: 0.3966INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 6s 5ms/step - loss: 1.6700 - accuracy: 0.3965 - val_loss: 1.6814 - val_accuracy: 0.3987
Epoch 14/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6526 - accuracy: 0.4034 - val_loss: 1.7092 - val_accuracy: 0.3853
Epoch 15/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6363 - accuracy: 0.4108 - val_loss: 1.7020 - val_accuracy: 0.3939
Epoch 16/20
1249/1250 [============================>.] - ETA: 0s - loss: 1.6262 - accuracy: 0.4193INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 6s 5ms/step - loss: 1.6263 - accuracy: 0.4193 - val_loss: 1.6723 - val_accuracy: 0.4002
Epoch 17/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6164 - accuracy: 0.4195 - val_loss: 1.6974 - val_accuracy: 0.3934
Epoch 18/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6062 - accuracy: 0.4241 - val_loss: 1.6958 - val_accuracy: 0.4029
Epoch 19/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6004 - accuracy: 0.4244 - val_loss: 1.6914 - val_accuracy: 0.3939
Epoch 20/20
1250/1250 [==============================] - 6s 4ms/step - loss: 1.5862 - accuracy: 0.4314 - val_loss: 1.7306 - val_accuracy: 0.3850


### Exercise 8.c

In [73]:
input_shape = list(X_train.shape[1:])
optimizer = tf.keras.optimizers.legacy.Nadam()
model = build_dnn_model(input_shape, 20, 100, 10, optimizer, "sparse_categorical_crossentropy", use_batch_norm=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model", save_best_only=True)

history = fit_dnn_model(model, X_train, y_train, epochs=20, validation_split=0.2, 
                        callbacks=[early_stopping_cb, model_checkpoint_cb])

Epoch 1/20
1244/1250 [============================>.] - ETA: 0s - loss: 2.0459 - accuracy: 0.2563INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 13s 8ms/step - loss: 2.0449 - accuracy: 0.2567 - val_loss: 1.8558 - val_accuracy: 0.3309
Epoch 2/20
1250/1250 [==============================] - ETA: 0s - loss: 1.7872 - accuracy: 0.3584INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 9s 7ms/step - loss: 1.7872 - accuracy: 0.3584 - val_loss: 1.7480 - val_accuracy: 0.3704
Epoch 3/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.7074 - accuracy: 0.3905 - val_loss: 1.7833 - val_accuracy: 0.3753
Epoch 4/20
1249/1250 [============================>.] - ETA: 0s - loss: 1.6600 - accuracy: 0.4052INFO:tensorflow:Assets written to: my_cifar10_model/assets


...

1250/1250 [==============================] - 9s 7ms/step - loss: 1.5639 - accuracy: 0.4421 - val_loss: 1.6219 - val_accuracy: 0.4191
Epoch 7/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.5265 - accuracy: 0.4568 - val_loss: 1.6880 - val_accuracy: 0.3991
Epoch 8/20
1242/1250 [============================>.] - ETA: 0s - loss: 1.4806 - accuracy: 0.4734INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 10s 8ms/step - loss: 1.4805 - accuracy: 0.4732 - val_loss: 1.5304 - val_accuracy: 0.4427
Epoch 9/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.4496 - accuracy: 0.4854 - val_loss: 1.7085 - val_accuracy: 0.4062
Epoch 10/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.4106 - accuracy: 0.5002 - val_loss: 1.5543 - val_accuracy: 0.4531
Epoch 11/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.3771 - accuracy: 0.5128 - val_loss: 1.6876 - val_accuracy: 0.4261
Epoch 12/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.3518 - accuracy: 0.5230 - val_loss: 1.5897 - val_accuracy: 0.4335
Epoch 13/20
1242/1250 [============================>.] - ETA: 0s - loss: 1.3253 - accuracy: 0.5336INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 10s 8ms/step - loss: 1.3253 - accuracy: 0.5335 - val_loss: 1.4478 - val_accuracy: 0.4888
Epoch 14/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2886 - accuracy: 0.5444 - val_loss: 1.7300 - val_accuracy: 0.4241
Epoch 15/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2662 - accuracy: 0.5551 - val_loss: 1.5246 - val_accuracy: 0.4626
Epoch 16/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2457 - accuracy: 0.5609 - val_loss: 1.5252 - val_accuracy: 0.4761
Epoch 17/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2165 - accuracy: 0.5685 - val_loss: 1.5431 - val_accuracy: 0.4585
Epoch 18/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2025 - accuracy: 0.5777 - val_loss: 1.6127 - val_accuracy: 0.4454


In [78]:
input_shape = list(X_train.shape[1:])
optimizer = tf.keras.optimizers.legacy.Nadam()
model = build_dnn_model(input_shape, 20, 100, 10, optimizer, "sparse_categorical_crossentropy", 
                        use_batch_norm=True, use_dropout=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model", save_best_only=True)

history = fit_dnn_model(model, X_train, y_train, epochs=20, validation_split=0.2, 
                        callbacks=[early_stopping_cb, model_checkpoint_cb])

Epoch 1/20
1244/1250 [============================>.] - ETA: 0s - loss: 2.1958 - accuracy: 0.2100INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 13s 9ms/step - loss: 2.1949 - accuracy: 0.2104 - val_loss: 2.1575 - val_accuracy: 0.2647
Epoch 2/20
1243/1250 [============================>.] - ETA: 0s - loss: 1.8514 - accuracy: 0.3345INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 10s 8ms/step - loss: 1.8503 - accuracy: 0.3346 - val_loss: 1.8648 - val_accuracy: 0.3221
Epoch 3/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.7283 - accuracy: 0.3837 - val_loss: 1.9117 - val_accuracy: 0.3171
Epoch 4/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.6630 - accuracy: 0.4059 - val_loss: 1.9100 - val_accuracy: 0.3441
Epoch 5/20
1248/1250 [============================>.] - ETA: 0s - loss: 1.6174 - accuracy: 0.4211INFO:tensorflow:Assets written to: my_cifar10_model/assets


...

1250/1250 [==============================] - 10s 8ms/step - loss: 1.4225 - accuracy: 0.4957 - val_loss: 1.5870 - val_accuracy: 0.4477
Epoch 11/20
1247/1250 [============================>.] - ETA: 0s - loss: 1.3887 - accuracy: 0.5121INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 11s 8ms/step - loss: 1.3886 - accuracy: 0.5121 - val_loss: 1.5601 - val_accuracy: 0.4546
Epoch 12/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.3628 - accuracy: 0.5194 - val_loss: 1.6497 - val_accuracy: 0.4318
Epoch 13/20
1243/1250 [============================>.] - ETA: 0s - loss: 1.3332 - accuracy: 0.5302INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1250/1250 [==============================] - 10s 8ms/step - loss: 1.3335 - accuracy: 0.5300 - val_loss: 1.5093 - val_accuracy: 0.4736
Epoch 14/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.3050 - accuracy: 0.5415 - val_loss: 1.6229 - val_accuracy: 0.4421
Epoch 15/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.2844 - accuracy: 0.5483 - val_loss: 1.6597 - val_accuracy: 0.4607
Epoch 16/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.2597 - accuracy: 0.5609 - val_loss: 1.5359 - val_accuracy: 0.4680
Epoch 17/20
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2410 - accuracy: 0.5647 - val_loss: 1.5774 - val_accuracy: 0.4668
Epoch 18/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.2189 - accuracy: 0.5695 - val_loss: 1.5896 - val_accuracy: 0.4736


In [86]:
# MC dropout
# Training=True ensures that the dropout layer remains active, ensuring different predictions each time
y_probas = np.stack([model(X_test, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_pred = np.argmax(y_proba, axis=1)

In [91]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.5207